# Visualizing the Drug Epidemic in San Francisco 

<p>San Francisco, a city once celebrated for its vibrant and diverse cultures, has faced numerous challenges in recent years, particularly with regards to <b>drug addiction and crime</b>.</p>


In response to these issues, the city has implemented several strategies aimed at addressing drug addiction and its impact on the community. These strategies include <b>increasing law enforcement efforts</b> to combat illegal drug possession and sales, as well as <b>deploying more prescription medications</b>, such as Narcan, to reverse overdose and save lives.

> Narcan, also known as naloxone, is a prescription medication used to reverse opioid overdose. It works by binding to the same receptors in the brain that opioids bind to, reversing their effects and restoring breathing.

Drug overdose in San Francisco has been driven largely by the proliferation of the synthetic opioid fentanyl, according to the [SF Chronicle](https://www.sfchronicle.com/sf/article/S-F-is-investing-millions-in-overdose-response-16491976.php). On Dec. 17, 2021, Mayor London Breed declared an official state of emergency in the Tenderloin district to address the escalating epidemic.

Using the [San Francisco police response records](https://data.sfgov.org/Public-Safety/Police-Department-Incident-Reports-2018-to-Present/wg3w-h783), we hope to answer the following questions:

- Where do most drug-related arrests and Narcan deployments occur?
- How do the frequencies change over time?
- When is Narcan most commonly deployed? Mornings? Evenings? Late at night?
- How about drug-related arrests?

### Downloading and Preparing Data 

The SFPD data can be downloaded by clicking `Export` - `CSV`:

![export-csv](data_download.png)

Save the dataset to `data/raw`. 

> Notice that since this file is big (over 200MB), before committing and pushing to github, make sure to add the line '/data' in `.gitignore` file so that we don't break the git!

In [4]:
# importing the modules 
import csv
import pandas as pd
import altair as alt

In [5]:
# Reading the file from local data/raw folder
sfpd_data = pd.read_csv("/Users/tracy/sfpd-data/data/raw/Police_Department_Incident_Reports__2018_to_Present.csv")

In [7]:
# Filtering for narcan deployment incidents
narcan_deployment = sfpd_data[sfpd_data['Incident Code'] == 51050]

In [4]:
# Filtering for drug-related arrests
drug_arrest = sfpd_data[~sfpd_data['Incident Category'].isna()]
drug_arrest_data = drug_arrest[drug_arrest['Incident Category'].str.contains('Drug')]

In [ ]:
# Saving the files to the data/processed directory
narcan_deployment.to_csv("/Users/tracy/sfpd-data/data/processed/narcan_deployment.csv")
drug_arrest_data.to_csv("/Users/tracy/sfpd-data/data/processed/drug_arrest_data.csv")

Now, load the filterd csv files and be ready to dive in!

In [15]:
# Read dataframes
drug_arrest = pd.read_csv('/Users/tracy/code/sfpd_data/data/processed/drug_arrest_data.csv')
narcan_deploy = pd.read_csv('/Users/tracy/sfpd-data/data/processed/narcan_deployment.csv')
# Make the column names snake_case
drug_arrest.columns = [col.lower().replace(' ', '_') for col in drug_arrest.columns]
narcan_deploy.columns = [col.lower().replace(' ', '_') for col in narcan_deploy.columns]

### Drug Arrests and Narcan Deployment by Neighborhood

#### Finding #1
Our analysis found that most drug-related arrests occur in the **Tenderloin, South of Market (SoMa), and Mission** neighborhoods. Located near each other in downtown San Francisco, the two neighborhoods have undergone significant changes following rapid urban development and gentrification in recent years. Both suffer from a high-rate of poverty and crime.

#### Analysis and Visualization

In [91]:
da_neighb = drug_arrest.groupby(drug_arrest['analysis_neighborhood']).size()
da_df = da_neighb.sort_values(ascending = False).reset_index(name='drug_arrests_count')
da_df.index +=1 
da_df.head(10)

,analysis_neighborhood,drug_arrests_count
1,Tenderloin,8613
2,South of Market,2962
3,Mission,1740
4,Financial District/South Beach,451
5,Bayview Hunters Point,437
6,Castro/Upper Market,291
7,Western Addition,273
8,Haight Ashbury,242
9,Nob Hill,213
10,North Beach,137


In [42]:
# Group the incidents by time and location
# Note that incident_date_2 has no significant meaning other than for the purpose of tooltips in visualization.

neighborho = drug_arrest.groupby([(drug_arrest['incident_date'].dt.strftime('%Y-%m')),drug_arrest['analysis_neighborhood']]).size().reset_index(name='count')
neighborho['incident_date_2'] = neighborho['incident_date']
sorted = neighborho.sort_values('count',ascending=False)

In [69]:
# Visualization
alt.Chart(sorted).mark_area(size=9,opacity=0.8).encode(
    x= alt.X('incident_date:T',title='Date',axis=alt.Axis(domain=False, format='%Y.%m')),
    y= alt.Y('count:Q',stack = 'normalize',title = 'Count'),
    color=alt.Color("analysis_neighborhood",legend=alt.Legend(title='Neighborhood', values=['Tenderloin', 'South of Market','Mission'])),
    tooltip = [alt.Tooltip('incident_date_2',title = 'Date'),alt.Tooltip('count',title='Drug Arrests'),alt.Tooltip('analysis_neighborhood',title='Neighborhood')]
).properties(
    width=650,
    height=350,
    title='Total Drug Arrests by Neighborhoods (2018-now)'
)

alt.Chart(...)

From 2018, the number of drug-arrests in Tenderloin along is more than that in all the other neighborhoods combined.

In [81]:
# Outside of tenderloin
other_neighbor = sorted.loc[sorted['analysis_neighborhood'] != 'Tenderloin']
grouped_o = other_neighbor.groupby(['incident_date'])['count'].sum().reset_index()
# Tenderloin
tenderloin = sorted.loc[sorted['analysis_neighborhood'] == 'Tenderloin']
grouped_t = tenderloin.groupby(['incident_date'])['count'].sum().reset_index()
# Compare
compare = pd.DataFrame({
    'incident_date':grouped_t['incident_date'],
    'tenderloin_incidents': grouped_t['count'],
    'other_incidents':grouped_o['count']
})
totals = compare.drop('incident_date', axis=1).sum().to_frame().T
totals.index = ['Totals']
totals

,tenderloin_incidents,other_incidents
Totals,8613,8610


In [101]:
# Visualization
tenderloin_line = alt.Chart(compare).mark_line(color = 'crimson',opacity=0.6).encode(
    alt.X('incident_date:T',title='Date'),
    alt.Y('tenderloin_incidents:Q',title = 'Count'))

others_line = alt.Chart(compare).mark_line(color = 'grey').encode(
    x= alt.X('incident_date:T',title='Date'),
    y= alt.Y('other_incidents:Q',title = 'Count'))

(tenderloin_line + others_line).properties(
    width=600,
    height=300,
    title='Drug-related Arrests in Tenderloin vs. All Other Neighborhoods'
)

alt.LayerChart(...)

#### Finding #2

A similar pattern is found with Narcan deployment:<br>
incidents happen significantly more often in **Terderloin** than in other neighborhoods.

In [92]:
nd_neighb = narcan_deploy.groupby(narcan_deploy['analysis_neighborhood']).size()
nd_df = nd_neighb.sort_values(ascending = False).reset_index(name='narcan_deployment_count')
nd_df.index +=1 
nd_df.head(10)

,analysis_neighborhood,narcan_deployment_count
1,Tenderloin,401
2,South of Market,153
3,Financial District/South Beach,56
4,Mission,38
5,Bayview Hunters Point,15
6,Castro/Upper Market,12
7,Western Addition,10
8,Marina,7
9,Chinatown,7
10,North Beach,6


In [ ]:
# Outside of tenderloin
other_neighbor = sorted.loc[sorted['analysis_neighborhood'] != 'Tenderloin']
grouped_o = other_neighbor.groupby(['incident_date'])['count'].sum().reset_index()
# Tenderloin
tenderloin = sorted.loc[sorted['analysis_neighborhood'] == 'Tenderloin']
grouped_t = tenderloin.groupby(['incident_date'])['count'].sum().reset_index()
# Compare
compare = pd.DataFrame({
    'incident_date':grouped_t['incident_date'],
    'tenderloin_incidents': grouped_t['count'],
    'other_incidents':grouped_o['count']
})
totals = compare.drop('incident_date', axis=1).sum().to_frame().T
totals.index = ['Totals']
totals

<h3>Drug Arrests and Narcan Deployment Over Time</h3>

#### Finding #3
As the pandemic began to subside and people started moving around more freely in the city, there has been an increase in drug-related arrests. During July and August of 2022, the city saw **more than 1000** drug-related arrests, almost double the typical monthly average of **271**.

In [103]:
# Drug arrests over time, by year-month
# Again, note that incident_date_2 has no significant meaning other than for the purpose of tooltips in visualization.
drug_arrest['incident_date'] = pd.to_datetime(drug_arrest['incident_date'])
drug_arrest['incident_time'] = pd.to_datetime(drug_arrest['incident_time'])

counts = drug_arrest.groupby(drug_arrest['incident_date'].dt.strftime('%Y-%m')).size().reset_index(name='count')
counts['incident_date_2'] = counts['incident_date']
counts.head()

,incident_date,count,incident_date_2
0,2018-01,337,2018-01
1,2018-02,356,2018-02
2,2018-03,423,2018-03
3,2018-04,348,2018-04
4,2018-05,384,2018-05


In [108]:
# Excluding the month we're currently in.
end_date = '2023-02'
filtered_counts = counts[counts['incident_date']<=end_date]
# Calculating the monthly average. We chose median to account for extreme values.
drug_arrest_median = filtered_counts['count'].median()
drug_arrest_median

271.5

In [131]:
# Visualize
base = alt.Chart(filtered_counts).mark_line(size=4).encode(
    alt.X('incident_date:T',title='Date'),
    alt.Y('count',title='Count',scale=alt.Scale(domain=(0,600))),
    tooltip = [alt.Tooltip('incident_date_2',title = 'Date'),alt.Tooltip('count',title='Drug Arrests')]
).properties(
    width=600,
    height=400,
    title='Drug-related Arrests, Jan. 2018 - Feb.2023'
)

median = alt.Chart(filtered_counts).mark_rule(size=2,color='crimson').encode(
    y='median(count)',
    tooltip = alt.Tooltip(title = 'Median of counts')
)

text = (
    alt.Chart(filtered_counts.query("count == count.max()"))
    .mark_text(dy=-15, color="crimson")
    .encode(x=alt.X("incident_date:T"), y=alt.Y("count"), text=alt.Text(("count")) 
))
base+median+text

alt.LayerChart(...)

#### Finding #4
Narcan deployment spiked as incidents of drug overdose worsened during the COVID-19 pandemic.

In March 2020, there were **31 recorded deployments** of Narcan in San Francisco, more than doubling the typical monthly amount of **12**.


In [132]:
# Grouping the narcan deployments by Year-month.
# Note that incident_date_2 is not necessary, just for the sake of tooltips for visualization
narcan_deploy['incident_date'] = pd.to_datetime(narcan_deploy['incident_date'])
narcan_deploy['incident_time'] = pd.to_datetime(narcan_deploy['incident_time'])

nd_counts = narcan_deploy.groupby(narcan_deploy['incident_date'].dt.strftime('%Y-%m')).size().reset_index(name='count')
nd_counts['incident_date_2'] = nd_counts['incident_date']
nd_counts.head()

,incident_date,count,incident_date_2
0,2018-01,4,2018-01
1,2018-02,13,2018-02
2,2018-03,7,2018-03
3,2018-04,5,2018-04
4,2018-05,3,2018-05


In [135]:
# Excluding the month we're currently in.
end_date = '2023-02'
filtered_nd_counts = nd_counts[nd_counts['incident_date']<=end_date]
# Calculating the monthly average. We chose median to account for extreme values.
median_nd = filtered_nd_counts['count'].median()
median_nd

12.0

In [140]:
# Visualize
base = alt.Chart(filtered_nd_counts).mark_line(size=4).encode(
    alt.X('incident_date:T',title='Date'),
    alt.Y('count',title = 'Counts'),
    tooltip = [alt.Tooltip('incident_date_2',title = 'Date'),alt.Tooltip('count',title='Drug Arrests')]
).properties(
    width=600,
    height=400,
    title='Narcan Deployments, Jan 2018 - Feb 2023'
)
median = alt.Chart(filtered_nd_counts).mark_rule(size=2,color='crimson').encode(
    y='median(count)',
    tooltip = alt.Tooltip(title = 'Median of counts')
)
text = (
    alt.Chart(filtered_nd_counts.query("count == count.max()"))
    .mark_text(dy=-25, color="crimson")
    .encode(x=alt.X("incident_date:T"), y=alt.Y("count"), text=alt.Text(("count")) 
))
base+median+text

alt.LayerChart(...)

### Drug Arrests by Day-of-Week and Time-of-Day

#### Finding #5

Most drug-related arrests occur on the **afternoons of Tuesday and Wednesday**. This result could be due to routine police patrol around those times.

In [142]:
# Grouping the arrests
week_time = drug_arrest.groupby([drug_arrest['incident_day_of_week'],drug_arrest['incident_time'].dt.strftime('%H')]).size().reset_index(name='count')
week_time['hour']=week_time['incident_time']+':00'
week_time.head()

,incident_day_of_week,incident_time,count,hour
0,Friday,00,73,00:00
1,Friday,01,37,01:00
2,Friday,02,23,02:00
3,Friday,03,33,03:00
4,Friday,04,22,04:00


In [144]:
base = alt.Chart(week_time).mark_rect().encode(
    alt.Y('incident_day_of_week',title = 'Day of Week'),
    alt.X('incident_time',title = 'Hour of Day'),
    color='count',
    tooltip = [alt.Tooltip('incident_day_of_week',title='Day'),
               alt.Tooltip('hour',title = 'Time'),
               alt.Tooltip('count',title='Drug Arrests')]
)
base

alt.Chart(...)

## Beyond the Notebook

Explore our dashboard to see how the drug epidemic affects different neighborhoods in San Francisco.